In [32]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models import LdaMulticore

from pprint import pprint

import pickle
import re
import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
import pandas as pd

In [33]:
df = pd.read_csv("data/all_tweets_with_sentiment.csv")
df

,Unnamed: 0,User,Time,Likes,Source,Tweet,cleaned_tweet,sent_analysis
0,11,sidney_ella,2009-12-28 21:37:33+00:00,0.0,Twitter Web Client,@heartsandhandss I think women who choose natu...,@user I think women who choose natural are lik...,"{'label': 'positive', 'score': 0.8187388181686..."
1,15,DesirreAndrews,2009-12-28 00:01:42+00:00,0.0,Twitter Web Client,"@academicobgyn Perhaps with education, suppor...","@user Perhaps with education, support and nat...","{'label': 'neutral', 'score': 0.7713251709938049}"
2,21,foreveradog,2009-12-25 02:35:57+00:00,0.0,Twitter Web Client,Natural Labor Induction Induce Labor Naturally...,Natural Labor Induction Induce Labor Naturally...,"{'label': 'neutral', 'score': 0.8671206831932068}"
3,23,lvonestopshop,2009-12-24 17:25:21+00:00,0.0,Twitter Web Client,Natural Way to Induce Labor - It Works! http:/...,Natural Way to Induce Labor - It Works! http,"{'label': 'positive', 'score': 0.8338969349861..."
4,28,lvonestopshop,2009-12-23 09:18:20+00:00,0.0,Twitter Web Client,Use Acupressure for Labor For Natural Onset of...,Use Acupressure for Labor For Natural Onset of...,"{'label': 'neutral', 'score': 0.9261870980262756}"
...,...,...,...,...,...,...,...,...
717208,658396,nomina_bot,2018-01-01 02:30:46+00:00,0.0,twittbot.net,硬膜上腔：epidural space #nomina,硬膜上腔：epidural space #nomina,"{'label': 'neutral', 'score': 0.9026492238044739}"
717209,658397,WrightByrdYT,2018-01-01 01:28:34+00:00,0.0,Twitter for Android,@Fact Well yeah if you don't have an epidural,@user Well yeah if you don't have an epidural,"{'label': 'neutral', 'score': 0.7822378873825073}"
717210,658398,MorganShaw513,2018-01-01 01:09:31+00:00,1.0,Twitter for iPhone,@lynkrystal Yeah I’m definitely terrified of h...,@user Yeah I’m definitely terrified of having ...,"{'label': 'negative', 'score': 0.6401897072792..."
717211,658399,TehAngryAnalyst,2018-01-01 00:59:17+00:00,3.0,Twitter for iPhone,@ThomasKlineMD Managed care was a factor in th...,@user Managed care was a factor in this “crisi...,"{'label': 'negative', 'score': 0.7652140259742..."


In [34]:
import string
df['cleaned_tweet'] = [i.lower() for i in df['cleaned_tweet']]

df = df[df["cleaned_tweet"].str.contains("steroid") == False]
df = df[df["cleaned_tweet"].str.contains("stimulator") == False]
df = df[df["cleaned_tweet"].str.contains("surgery") == False]
df = df[df["cleaned_tweet"].str.contains("block") == False]
df = df[df["cleaned_tweet"].str.contains("http") == False]
df = df[df["cleaned_tweet"].str.contains("https") == False]


df['cleaned_tweet'] = [i.replace('\n', ' ')for i in df['cleaned_tweet']]
df['cleaned_tweet'] = [i.replace('\t', ' ')for i in df['cleaned_tweet']]
#df['cleaned_tweet'] = [i.replace(' rt', '')for i in df['cleaned_tweet']]
df['cleaned_tweet'] = [i.replace('http', '')for i in df['cleaned_tweet']]
df['cleaned_tweet'] = [i.replace('https', '')for i in df['cleaned_tweet']]
df

,Unnamed: 0,User,Time,Likes,Source,Tweet,cleaned_tweet,sent_analysis
0,11,sidney_ella,2009-12-28 21:37:33+00:00,0.0,Twitter Web Client,@heartsandhandss I think women who choose natu...,@user i think women who choose natural are lik...,"{'label': 'positive', 'score': 0.8187388181686..."
1,15,DesirreAndrews,2009-12-28 00:01:42+00:00,0.0,Twitter Web Client,"@academicobgyn Perhaps with education, suppor...","@user perhaps with education, support and nat...","{'label': 'neutral', 'score': 0.7713251709938049}"
5,38,dianapeiwu,2009-12-19 16:15:23+00:00,0.0,Twitter Web Client,the wealth of north america & eur comes from t...,the wealth of north america & eur comes from t...,"{'label': 'negative', 'score': 0.5011822581291..."
6,39,MoveGen,2009-12-19 16:15:23+00:00,0.0,Twitter Web Client,the wealth of north america & eur comes from t...,the wealth of north america & eur comes from t...,"{'label': 'negative', 'score': 0.5011822581291..."
23,104,emilyrdickey,2009-12-05 01:52:13+00:00,0.0,Twitter Web Client,@Wonderkarin the Bradley method is husband coa...,@user the bradley method is husband coached bi...,"{'label': 'neutral', 'score': 0.9033392667770386}"
...,...,...,...,...,...,...,...,...
717207,658395,fiercekittenz,2018-01-01 03:07:46+00:00,1.0,Twitter Web Client,@DallasNChains @christawatson @katemorris Spea...,@user @user @user speaking as someone who had ...,"{'label': 'negative', 'score': 0.4943246245384..."
717208,658396,nomina_bot,2018-01-01 02:30:46+00:00,0.0,twittbot.net,硬膜上腔：epidural space #nomina,硬膜上腔：epidural space #nomina,"{'label': 'neutral', 'score': 0.9026492238044739}"
717209,658397,WrightByrdYT,2018-01-01 01:28:34+00:00,0.0,Twitter for Android,@Fact Well yeah if you don't have an epidural,@user well yeah if you don't have an epidural,"{'label': 'neutral', 'score': 0.7822378873825073}"
717210,658398,MorganShaw513,2018-01-01 01:09:31+00:00,1.0,Twitter for iPhone,@lynkrystal Yeah I’m definitely terrified of h...,@user yeah i’m definitely terrified of having ...,"{'label': 'negative', 'score': 0.6401897072792..."


In [35]:
docs = df['cleaned_tweet'].tolist()
docs

['@user i think women who choose natural are like super heroes! especially since giving birth. epi though. caved bc of back labor.',
 '@user  perhaps with education, support and natural birth skills prior to labor more would opt in.',
 'the wealth of north america & eur comes from the labor, death & robbing of natural resources of the global south.  tf, climate debt. #cop15',
 'the wealth of north america & eur comes from the labor, death & robbing of natural resources of the global south.  tf, climate debt. #cop15',
 '@user the bradley method is husband coached birth- where he takes the lead in guiding you for a natural labor',
 'my sister wanted to do a natural birth but she has now been in labor for 7 hours and asked for an epidural...i know she couldnt handle it ha',
 '#haveababybyme u betta have health insurance cuz a natural birth and whatchu want...i was a 10lb baby bout killed my mom for 18hrs of labor',
 'my co-workers wife just gave natural birth to a 9.5 pound baby this am a

In [36]:
all_stopwords = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]

In [37]:
from tqdm import tqdm
docs = [i.translate(str.maketrans('', '', string.punctuation)) for i in docs]
for i,doc in tqdm(enumerate(docs)):
    docs[i] = [token for token in doc.split() if token not in all_stopwords]

print(docs[0:10])

548758it [01:27, 6266.06it/s]

[['user', 'women', 'choose', 'natural', 'super', 'heroes', 'birth', 'epi', 'caved', 'labor'], ['user', 'education', 'support', 'natural', 'birth', 'skills', 'prior', 'labor', 'opt'], ['wealth', 'north', 'america', 'eur', 'labor', 'death', 'robbing', 'natural', 'resources', 'global', 'south', 'climate', 'debt', 'cop15'], ['wealth', 'north', 'america', 'eur', 'labor', 'death', 'robbing', 'natural', 'resources', 'global', 'south', 'climate', 'debt', 'cop15'], ['user', 'bradley', 'method', 'husband', 'coached', 'birth', 'takes', 'lead', 'guiding', 'natural', 'labor'], ['sister', 'wanted', 'natural', 'birth', 'labor', '7', 'hours', 'asked', 'epidurali', 'handle', 'ha'], ['haveababybyme', 'betta', 'health', 'insurance', 'cuz', 'natural', 'birth', 'whatchu', 'wanti', '10lb', 'baby', 'bout', 'killed', 'mom', '18hrs', 'labor'], ['coworkers', 'wife', 'natural', 'birth', '95', 'pound', 'baby', '16', 'days', 'late', '52', 'hours', 'labor', 'yikes'], ['user', 'didnt', 'scar', 'tissue', '39', 'wks',

In [38]:
# Create Dictionary
id2word = corpora.Dictionary(docs)
corpus = [id2word.doc2bow(text) for text in docs]
print(corpus[:3])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)], [(0, 1), (5, 1), (6, 1), (8, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(5, 1), (6, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]]


In [39]:
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:1]]

[[('birth', 1),
  ('caved', 1),
  ('choose', 1),
  ('epi', 1),
  ('heroes', 1),
  ('labor', 1),
  ('natural', 1),
  ('super', 1),
  ('user', 1),
  ('women', 1)]]

In [40]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=42,
                                           chunksize=10000,
                                           passes=50,
                                           per_word_topics=True)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.051*"epidural" + 0.042*"user" + 0.005*"birth" + 0.004*"hematoma" + '
  '0.004*"natural" + 0.004*"12" + 0.003*"gas" + 0.003*"labour" + 0.003*"sakit" '
  '+ 0.003*"labor"'),
 (1,
  '0.106*"epidural" + 0.091*"user" + 0.015*"birth" + 0.015*"pain" + '
  '0.011*"natural" + 0.010*"labor" + 0.009*"lol" + 0.008*"baby" + 0.008*"amp" '
  '+ 0.007*"time"'),
 (2,
  '0.069*"user" + 0.061*"epidural" + 0.012*"una" + 0.011*"sin" + 0.010*"por" + '
  '0.010*"parto" + 0.009*"para" + 0.009*"mi" + 0.009*"yo" + 0.008*"las"')]


In [41]:
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis


/home/karl/mambaforge/envs/epidural/lib/python3.8/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.194957  0.160862       1        1  63.579399
2     -0.285553  0.044631       2        1  20.401480
0      0.090596 -0.205493       3        1  16.019121, topic_info=          Term           Freq          Total Category  logprob  loglift
8         user  450518.000000  450518.000000  Default  30.0000  30.0000
195   epidural  503011.000000  503011.000000  Default  29.0000  29.0000
1626       una   13797.000000   13797.000000  Default  28.0000  28.0000
4343       sin   13380.000000   13380.000000  Default  27.0000  27.0000
178       pain   54416.000000   54416.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
5        labor    2683.117449   37298.447173   Topic3  -5.8311  -0.8006
39       hours    2191.347939   16836.673161   Topic3  -6.0336  -0.2077
227   hospital    1721.542486   13462.170902   Topic3  -6.2749  -0.2253
283          2    1632.339265   14124.597258   Topic3  -6.3281  -0.3265
351        amp    1676.506531   29277.067873   Topic3  -6.3014  -1.0287

[187 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
235        1  0.565522     1
235        2  0.116156     1
235        3  0.318301     1
222        1  0.114898    12
222        2  0.087849    12
...      ...       ...   ...
23425      3  0.999618  yang
7688       3  0.999621    yg
4385       1  0.000188    yo
4385       2  0.999810    yo
21760      3  0.999368    za

[273 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [42]:
# Compute Coherence Score

coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('nCoherence Score: ', coherence_lda)

nCoherence Score:  0.5027092201388796


In [43]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -7.796004851517167


In [44]:
import numpy as np
def get_max(doc):
        idx,l = zip(*doc)
        return idx[np.argmax(l)]


l = [get_max(doc) for doc in lda_model.get_document_topics(corpus)]
print(l[0:10])

[1, 0, 0, 0, 1, 1, 1, 1, 1, 1]


In [45]:
print(l[0:100])

[1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [46]:
print(l[100:200])

[1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 2, 2, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1]


In [47]:
df['topic'] = l
df.head()

,Unnamed: 0,User,Time,Likes,Source,Tweet,cleaned_tweet,sent_analysis,topic
0,11,sidney_ella,2009-12-28 21:37:33+00:00,0.0,Twitter Web Client,@heartsandhandss I think women who choose natu...,@user i think women who choose natural are lik...,"{'label': 'positive', 'score': 0.8187388181686...",1
1,15,DesirreAndrews,2009-12-28 00:01:42+00:00,0.0,Twitter Web Client,"@academicobgyn Perhaps with education, suppor...","@user perhaps with education, support and nat...","{'label': 'neutral', 'score': 0.7713251709938049}",0
5,38,dianapeiwu,2009-12-19 16:15:23+00:00,0.0,Twitter Web Client,the wealth of north america & eur comes from t...,the wealth of north america & eur comes from t...,"{'label': 'negative', 'score': 0.5011822581291...",0
6,39,MoveGen,2009-12-19 16:15:23+00:00,0.0,Twitter Web Client,the wealth of north america & eur comes from t...,the wealth of north america & eur comes from t...,"{'label': 'negative', 'score': 0.5011822581291...",0
23,104,emilyrdickey,2009-12-05 01:52:13+00:00,0.0,Twitter Web Client,@Wonderkarin the Bradley method is husband coa...,@user the bradley method is husband coached bi...,"{'label': 'neutral', 'score': 0.9033392667770386}",1


In [53]:
print(len(df))
filtered_df = df[df["topic"] == 1]
print(len(filtered_df))

548758
431524


In [54]:
filtered_df.to_csv("filtered_df.csv", index=False)

In [55]:
filtered_df

,Unnamed: 0,User,Time,Likes,Source,Tweet,cleaned_tweet,sent_analysis,topic
0,11,sidney_ella,2009-12-28 21:37:33+00:00,0.0,Twitter Web Client,@heartsandhandss I think women who choose natu...,@user i think women who choose natural are lik...,"{'label': 'positive', 'score': 0.8187388181686...",1
23,104,emilyrdickey,2009-12-05 01:52:13+00:00,0.0,Twitter Web Client,@Wonderkarin the Bradley method is husband coa...,@user the bradley method is husband coached bi...,"{'label': 'neutral', 'score': 0.9033392667770386}",1
29,123,Stile15,2009-12-01 18:43:35+00:00,0.0,Twitter Web Client,my sister wanted to do a natural birth but she...,my sister wanted to do a natural birth but she...,"{'label': 'negative', 'score': 0.7766241431236...",1
42,166,BenryHailey,2009-11-13 22:24:00+00:00,0.0,Twitter Web Client,#haveababybyme u betta have health insurance c...,#haveababybyme u betta have health insurance c...,"{'label': 'negative', 'score': 0.6303679347038...",1
48,194,bethany529,2009-11-09 00:56:08+00:00,0.0,Twitter Web Client,My co-workers wife just gave natural birth to ...,my co-workers wife just gave natural birth to ...,"{'label': 'negative', 'score': 0.8341954946517...",1
...,...,...,...,...,...,...,...,...,...
717205,658393,amil,2018-01-01 04:47:59+00:00,1.0,Twitter for iPhone,@chantalbraganza Damn. I had a 24 hr labour bu...,@user damn. i had a 24 hr labour but with the ...,"{'label': 'neutral', 'score': 0.37650564312934...",1
717207,658395,fiercekittenz,2018-01-01 03:07:46+00:00,1.0,Twitter Web Client,@DallasNChains @christawatson @katemorris Spea...,@user @user @user speaking as someone who had ...,"{'label': 'negative', 'score': 0.4943246245384...",1
717209,658397,WrightByrdYT,2018-01-01 01:28:34+00:00,0.0,Twitter for Android,@Fact Well yeah if you don't have an epidural,@user well yeah if you don't have an epidural,"{'label': 'neutral', 'score': 0.7822378873825073}",1
717210,658398,MorganShaw513,2018-01-01 01:09:31+00:00,1.0,Twitter for iPhone,@lynkrystal Yeah I’m definitely terrified of h...,@user yeah i’m definitely terrified of having ...,"{'label': 'negative', 'score': 0.6401897072792...",1


In [51]:
p = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(p, 'lda.html')

/home/karl/mambaforge/envs/epidural/lib/python3.8/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [52]:
lda_model.save('models/lda_model')